In [0]:
from google.colab import drive

GOOGLE_DRIVE_MOUNT_DIR = "/content/drive"
drive.mount(GOOGLE_DRIVE_MOUNT_DIR, force_remount=True)

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
import tensorflow as tf
import numpy as np
import keras.backend as K
import datetime, os
import matplotlib.pyplot as plt
import pandas as pd


from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras import losses, optimizers, activations
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.multiclass import unique_labels

# Ages classes

In [0]:
#dataset with 2 classes
TRAIN_PATH = "../content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT Binary Ages Dataset/train"
VAL_PATH = "../content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT Binary Ages Dataset/val"
TEST_PATH = "../content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT Binary Ages Dataset/test"

In [0]:
#dataset with 7 classes
TRAIN_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/FLIP Split Dataset Ages Balanced/train"
VAL_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/FLIP Split Dataset Ages Balanced/val"
TEST_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/FLIP Split Dataset Ages Balanced/test"

In [0]:
#dataset with 4 classes
TRAIN_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT UTK 4 Classes/train"
VAL_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT UTK 4 Classes/val"
TEST_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT UTK 4 Classes/test"

#Model Customization

In [0]:
#change number of classes according your configuration
CLASSES = 4

base_model = DenseNet201(weights='imagenet', include_top=False, input_shape = (200,200,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# training all the layers
for layer in base_model.layers:
  layer.trainable = True

filepath = "/content/drive/My Drive/Rede Neural/Checkpoints/Age Classification/DenseNet-weights.best-class-NEW-4classes.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]
      
model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
74842112/74836368 [==============================] - 3s 0us/step


In [0]:
#visualize trainable layers
for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)

In [0]:
model.summary()

#Data preparation

In [0]:
WIDTH = 200
HEIGHT = 200
BATCH_SIZE = 32


# data augmentation
train_datagen = ImageDataGenerator(rescale = 1./255)

val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(HEIGHT, WIDTH),
		batch_size=BATCH_SIZE,
		class_mode='categorical',
    shuffle=True)
    
validation_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

train_generator.class_indices


Found 11384 images belonging to 4 classes.
Found 1264 images belonging to 4 classes.


{'1-13': 0, '14-17': 1, '18-35': 2, '36-r': 3}

#Initializing Training

In [0]:
EPOCHS = 20
BATCH_SIZE = 32


history = model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks_list)

#Confusion Matrix

In [0]:
#for 7 classes
imgs_validation = 714

Y_pred = model.predict_generator(validation_generator, imgs_validation // BATCH_SIZE+1, workers=0)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1-10', '11-15', '16-18', '19-25', '26-40', '41-60', '61-r']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [0]:
#for 4 classes
imgs_validation = 1264

Y_pred = model.predict_generator(validation_generator, imgs_validation // BATCH_SIZE+1, workers=0)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1-12', '13-17', '18-35', '36-r']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [0]:
#for 2 classes
imgs_validation = 1000

Y_pred = model.predict_generator(validation_generator, imgs_validation // BATCH_SIZE+1, workers=0)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1-17', '18-r']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

#Plot History of training

In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Reload Model and Training process

In [0]:
model2 = load_model('/content/drive/My Drive/Rede Neural/Checkpoints/DenseNet-weights.best-class-newdivision-large.hdf5')

for layer in model2.layers:
  layer.trainable = True

model2.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model2.summary()

In [0]:
train_generator.reset()
validation_generator.reset()

In [0]:
EPOCHS = 10
BATCH_SIZE = 32

filepath = "/content/drive/My Drive/Rede Neural/Checkpoints/DenseNet-weights.best-class.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks = [checkpoint]


history = model2.fit_generator(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks)
  
NAME = "model-DenseNet-4-classes"  
model2.save(f"/content/drive/My Drive/Rede Neural/Models/Ages Classes/{NAME}.model")  

#Making Predictions

Reset test_generator for other prediction

In [0]:
test_generator.reset()

In [0]:
HEIGHT = 200
WIDTH = 200
BATCH_SIZE = 32

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)



In [0]:
model2 = load_model('/content/drive/My Drive/Rede Neural/Checkpoints/Age Classification/DenseNet-weights.best-class-NEW-4classes.hdf5')

num_images = 337

pred = model2.predict_generator(test_generator, verbose=1, steps=num_images/BATCH_SIZE)
predicted_class_indices = np.argmax(pred, axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames = test_generator.filenames
results = pd.DataFrame({"Filename":filenames,
                        "Predictions":predictions})

results
results.to_csv(r'/content/drive/My Drive/Rede Neural/TESTS/DenseNet201_test-4-classes.csv')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
11/10 [===============================] - 115s 10s/step


Save in CSV file

In [0]:
results.to_csv(r'/content/drive/My Drive/Rede Neural/TESTS/DenseNet201_test.csv')

#Reload best checkpoint and Confusion Matrix

In [0]:
model2 = load_model('/content/drive/My Drive/Rede Neural/Checkpoints/Age Classification/DenseNet-weights.best-class-NEW-4classes.hdf5')

Y_pred = model2.predict_generator(validation_generator, 1264 // BATCH_SIZE+1, workers=0)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1-13', '14-17', '18-35', '36-r']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))